In [1]:
import pandas as pd
import numpy as np

In [2]:
df_trx = pd.DataFrame({
    'Item': {0: ['Apple','Cola','Rice','Chicken'], 
             1: ['Apple','Cola','Rice'], 
             2: ['Apple','Cola'], 
             3: ['Apple','Pear'],
             4: ['Milk', 'Cola', 'Rice', 'Chicken'],
             5: ['Milk', 'Cola', 'Rice'],
             6: ['Milk', 'Cola'],
             7: ['Milk', 'Pear'],
            },
    'Invoice': {0: 't1', 
                1: 't2', 
                2: 't3', 
                3: 't4',
                4: 't5',
                5: 't6',
                6: 't7',
                7: 't8',
               },
})

# df_trx = pd.DataFrame({
#     'Item': {0: ['A','B','C'], 1: ['A','C'], 2: ['A','D'], 3: ['B','E','F']},
#     'Invoice': {0: 't1', 1: 't2', 2: 't3', 3: 't4'},
# })

df_trx

,Item,Invoice
0,"[Apple, Cola, Rice, Chicken]",t1
1,"[Apple, Cola, Rice]",t2
2,"[Apple, Cola]",t3
3,"[Apple, Pear]",t4
4,"[Milk, Cola, Rice, Chicken]",t5
5,"[Milk, Cola, Rice]",t6
6,"[Milk, Cola]",t7
7,"[Milk, Pear]",t8


In [3]:
minsup = 0.2 * df_trx.Item.count()
minsup

1.6

In [4]:
minconf = 0.7

In [5]:
# Grab the itemset
# k 1

item_invoice = df_trx['Item'].explode('Item')
item_invoice = item_invoice.to_list()
item_invoice

['Apple',
 'Cola',
 'Rice',
 'Chicken',
 'Apple',
 'Cola',
 'Rice',
 'Apple',
 'Cola',
 'Apple',
 'Pear',
 'Milk',
 'Cola',
 'Rice',
 'Chicken',
 'Milk',
 'Cola',
 'Rice',
 'Milk',
 'Cola',
 'Milk',
 'Pear']

In [6]:

def get_item_counter(combi_array, minsup):
    data_item = {}
    for c in combi_array:
        data_item[c] = (item_invoice.count(c))
        
    return data_item


def get_combination_table(trx_item_detail):

    # explode data
    df_trx_detail = df_trx.explode('Item')
    #print(df_trx_detail)
    
    data_item = df_trx_detail['Item'].unique()
    #print(data_item)


    df_c1 = get_item_counter(data_item, minsup)
    df_c1 = pd.DataFrame.from_dict(df_c1, orient='index')
    #print(df_c1)
    
    df_c1 = pd.DataFrame({
        'Item':df_c1.index,
        'Support':df_c1.values.flatten()
    })
    
    return df_c1

In [7]:
df_c1 = get_combination_table(item_invoice)
df_c1

,Item,Support
0,Apple,4
1,Cola,6
2,Rice,4
3,Chicken,2
4,Pear,2
5,Milk,4


In [8]:
def get_qualified_itemset(df_c1, minsup):
    return df_c1[df_c1['Support']>=minsup]

In [9]:
l1 = get_qualified_itemset(df_c1, minsup)
l1

,Item,Support
0,Apple,4
1,Cola,6
2,Rice,4
3,Chicken,2
4,Pear,2
5,Milk,4


In [10]:
def get_pruned_itemset(df_c1, minsup):
    return df_c1[df_c1['Support']<minsup]

In [11]:
get_pruned_itemset(df_c1, minsup)

,Item,Support


In [12]:
from itertools import combinations

def get_combination(itemset, k=1):
    return list(combinations(itemset, k))

In [13]:
def check_is_included(row_bucket_trx, c):
    
    counter = 0
    result = False
    #print(row_bucket_trx,c)
    
    try:
        for i in c:
            if row_bucket_trx.count(i) > 0:
                counter+=1
            
    except Exception as e:
        print(e, row_bucket_trx)

    #print("The counter: ", counter)
    
    if counter == len(c):
        result = True

    #print("The result: ",result)
    
    return result


def get_support_frequency(itemset_combination):
    print("Support Frequency")
    #print(itemset_combination)
    support = []
    for c in itemset_combination:
        #print(c)
        df_c = df_trx['Item'].apply(lambda x: check_is_included(x, c) )
        #print(df_c.value_counts().get(True, 0))
        support.append(df_c.value_counts().get(True, 0))
        #print("Sup : ",support)

    df_result = pd.DataFrame({
     'Item': itemset_combination,
     'Support': support  
    })
        
    return df_result

In [14]:
def get_new_combination(l1,k=2):
    c2 = get_combination(l1['Item'].values,k)
    df_c2 = get_support_frequency(c2)
    
    return df_c2

In [15]:
combi = []
k = 3
counter = 1

data_qualified = []

while counter<k:
    counter = counter+1

    print("\n==================================")
    print(f"\nApriori K={counter}")
    print("\n==================================")
    
    c = get_new_combination(l1,counter)
    print("Combination")
    print(c)
    l = get_qualified_itemset(c,minsup)
    data_qualified.append(l)
    print("\nQualified")
    print(l)
    p = get_pruned_itemset(c,minsup)
    print("\nPruned")
    print(p)

   



Apriori K=2

Support Frequency
Combination
                Item  Support
0      (Apple, Cola)        3
1      (Apple, Rice)        2
2   (Apple, Chicken)        1
3      (Apple, Pear)        1
4      (Apple, Milk)        0
5       (Cola, Rice)        4
6    (Cola, Chicken)        2
7       (Cola, Pear)        0
8       (Cola, Milk)        3
9    (Rice, Chicken)        2
10      (Rice, Pear)        0
11      (Rice, Milk)        2
12   (Chicken, Pear)        0
13   (Chicken, Milk)        1
14      (Pear, Milk)        1

Qualified
               Item  Support
0     (Apple, Cola)        3
1     (Apple, Rice)        2
5      (Cola, Rice)        4
6   (Cola, Chicken)        2
8      (Cola, Milk)        3
9   (Rice, Chicken)        2
11     (Rice, Milk)        2

Pruned
                Item  Support
2   (Apple, Chicken)        1
3      (Apple, Pear)        1
4      (Apple, Milk)        0
7       (Cola, Pear)        0
10      (Rice, Pear)        0
12   (Chicken, Pear)        0
13   (Chicken,

In [16]:
# Combination Rules

df_qualified = data_qualified[len(data_qualified)-1]
df_qualified

data_rules = []

for index,row in df_qualified.iterrows():
    items = np.asarray(row['Item'])
    rules = get_combination(items,len(items)-1)
    print(rules)
    

[('Apple', 'Cola'), ('Apple', 'Rice'), ('Cola', 'Rice')]
[('Cola', 'Rice'), ('Cola', 'Chicken'), ('Rice', 'Chicken')]
[('Cola', 'Rice'), ('Cola', 'Milk'), ('Rice', 'Milk')]


In [17]:
# Confidence Table

# Association rule
# Support
# Confidence
# Confidence %

In [18]:
l

,Item,Support
0,"(Apple, Cola, Rice)",2
10,"(Cola, Rice, Chicken)",2
12,"(Cola, Rice, Milk)",2


In [19]:
def get_rules(x):
    
    counter = 0
    result = ()
    res_dic = {}
    #print("item : ",x)

    try:
        c_set = get_combination(x, len(x)-1)
        #print("combi : ", c_set)
        for c in c_set:

            s_x = set(x)
            s_c = set(c)
            #print(list(s_x - s_c))
            res_dic[c]=list(s_x - s_c)
            
    except Exception as e:
        print(e)

    return res_dic

In [20]:
rules = l['Item'].apply(lambda x: get_rules(x))
type(rules)

pandas.core.series.Series

In [21]:

def get_rule_frequency(target_set):

    
    existense = 0
    
    for item in df_trx['Item']:
        
        counter = 0
        for t in target_set:
            if t in item:
                counter+=1

            if counter == len(target_set):
                existense += 1
                
    return existense

In [47]:
item_set = []
associated = []
occurence_set = []
occurence_target = []
confidence = []
confidence_percentage = []

for r in rules:
    #print(r)
    for r2 in r:
        #print(r2,"=>",r[r2])
        item_set.append(r2)
        associated.append(r[r2])

        #print(get_rule_frequency(r2))
        occ_set = get_rule_frequency(r2)
        occurence_set.append(occ_set)
        #print("====")
        complete_rule = list(np.array(r2))#,r[r2]]
        complete_rule.append(r[r2][0])
        #print(complete_rule)
        #print(get_rule_frequency(complete_rule))
        occ_target = get_rule_frequency(complete_rule)
        occurence_target.append(occ_target)
        confidence.append(occ_target/occ_set)
        confidence_percentage.append((occ_target/occ_set)*100)

df_confidence_table = pd.DataFrame({
    'item_sets' : item_set,
    'associated' : associated,
    'occurence' : occurence_set,
    'occurence_target' : occurence_target,
    'confidence' : confidence,
    'confidence_percentage' : confidence_percentage,
})

df_confidence_table

,item_sets,associated,occurence,occurence_target,confidence,confidence_percentage
0,"(Apple, Cola)",[Rice],3,2,0.666667,66.666667
1,"(Apple, Rice)",[Cola],2,2,1.000000,100.000000
2,"(Cola, Rice)",[Apple],4,2,0.500000,50.000000
3,"(Cola, Rice)",[Chicken],4,2,0.500000,50.000000
4,"(Cola, Chicken)",[Rice],2,2,1.000000,100.000000
5,"(Rice, Chicken)",[Cola],2,2,1.000000,100.000000
6,"(Cola, Rice)",[Milk],4,2,0.500000,50.000000
7,"(Cola, Milk)",[Rice],3,2,0.666667,66.666667
8,"(Rice, Milk)",[Cola],2,2,1.000000,100.000000


In [52]:
df_confidence_table[df_confidence_table['confidence']>=minconf]

,item_sets,associated,occurence,occurence_target,confidence,confidence_percentage
1,"(Apple, Rice)",[Cola],2,2,1.0,100.0
4,"(Cola, Chicken)",[Rice],2,2,1.0,100.0
5,"(Rice, Chicken)",[Cola],2,2,1.0,100.0
8,"(Rice, Milk)",[Cola],2,2,1.0,100.0
